In [46]:
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, MultiPolygon, Polygon
from shapely.ops import unary_union, split, linemerge, polygonize
import shapely as shp
import pandas as pd
import networkx as nx 
import re
import os
import numpy as np
os.chdir('/home/pedro/Documents/GIT_WORKSPACE/split_offdb')
os.getcwd()

'/home/pedro/Documents/GIT_WORKSPACE/split_offdb'

In [29]:
#Carregar grid
grid_gdf = gpd.read_parquet('inputs/grid.parquet')

# Carregar o arquivo de entrada
input_gdf = gpd.read_parquet('inputs/input.parquet')


In [32]:
unidade_split = grid_gdf[grid_gdf["grid_id"] == 187].geometry.values[0] 
gdf_input_intersection = input_gdf[input_gdf.intersects(unidade_split)]
gdf_input_intersection
  

,id,id_layer,geom
0,193206,CAR,"MULTIPOLYGON (((-44.25014 -2.65343, -44.24987 ..."
1,182433,CAR,"MULTIPOLYGON (((-44.23856 -2.65714, -44.23862 ..."
2,349580,CAR,"MULTIPOLYGON (((-44.24801 -2.68644, -44.24846 ..."
3,337735,CAR,"MULTIPOLYGON (((-44.19677 -2.66368, -44.19757 ..."
4,176811,CAR,"MULTIPOLYGON (((-44.25887 -2.66707, -44.25887 ..."
...,...,...,...
353346,249875,CAR,"MULTIPOLYGON (((-44.29129 -2.64609, -44.29125 ..."
353347,208346,CAR,"MULTIPOLYGON (((-44.27422 -2.6458, -44.27402 -..."
353348,288981,CAR,"MULTIPOLYGON (((-44.25486 -2.66914, -44.25484 ..."
353349,249899,CAR,"MULTIPOLYGON (((-44.29106 -2.64668, -44.29134 ..."


In [39]:
lines = []

for geom in gdf_input_intersection.geometry:
    merged_line = linemerge(geom.boundary)
    
    if isinstance(merged_line, MultiLineString):
        
        # Se for MultiLineString, combinar seus pontos em uma única LineString
        points = []
        for line in merged_line.geoms:
            points.extend(line.coords)  # Extrai coordenadas de cada LineString
        
        # Cria uma nova LineString com todos os pontos
        merged_line = LineString(points)
        
    lines.append(merged_line)
lines.append(unidade_split.boundary)

In [66]:
# Cria o MultiLineString a partir das linhas
multi_line = MultiLineString(lines) 
multi_line = gpd.GeoDataFrame(geometry=[multi_line], crs = "EPSG:4674")
multi_line.to_file('dev/multi_line.geojson', driver = 'GeoJSON')

multi_line = MultiLineString(lines) 

multi_line = gpd.GeoDataFrame(geometry=[multi_line], crs = "EPSG:4674")
multi_line.to_file('dev/multi_line_simplified.geojson', driver = 'GeoJSON')
# Usa shapely.node para adicionar nós (pontos de interseção) ao MultiLineString
#multi_line_with_nodes = shp.node(multi_line)

In [67]:
multi_line_with_nodes = shp.node(multi_line)